# COVID-Symptom Facebook Survey

In [ ]:
# Install packages locally
#!pip3 install --install-option="--prefix=$HOME/local" packagename

In [15]:
import numpy as np
import pandas as pd
import os
import re
import datetime as dt
import matplotlib.pyplot as plt
import plotnine
# Load all the locally installed packages
%cd /export/home/honlee/local/lib64/python3.6/site-packages
import sqlalchemy as sqla
import pymongo
%cd ../../../../
from loadEngine import *

/export/home/honlee/local/lib64/python3.6/site-packages
/export/home/honlee


In [2]:
# Check the files on the specified directory
for root, dirs, files in os.walk("data/survey-data"):
    for filename in files:
        print(filename)

OUTAGES.txt
2020-04.csv.gz
2020-04.tar
README.md
2020-05.csv.gz
2020-05.tar
2020-06.csv.gz
2020-06.tar
2020-07.csv.gz
2020-07.tar
2020-08.csv.gz
2020-08.tar
2020-09.csv.gz
2020-09.tar
2020-10.csv.gz
2020-10.tar
2020-11.csv.gz
2020-11.tar
2020-12.csv.gz
2021-01.csv.gz
cvid_responses_2020_04_10_recordedby_2020_05_30.csv.gz
cvid_responses_2020_04_06_recordedby_2020_05_30.csv.gz
cvid_responses_2020_04_10_recordedby_2020_06_03.csv.gz
cvid_responses_2020_04_06_recordedby_2020_06_03.csv.gz
cvid_responses_2020_04_10_recordedby_2020_09_03.csv.gz
cvid_responses_2020_04_06_recordedby_2020_09_03.csv.gz
cvid_responses_2020_04_10_recordedby_2020_12_09.csv.gz
cvid_responses_2020_04_06_recordedby_2020_12_09.csv.gz
cvid_responses_2020_04_11_recordedby_2020_05_30.csv.gz
cvid_responses_2020_04_07_recordedby_2020_05_30.csv.gz
cvid_responses_2020_04_11_recordedby_2020_06_03.csv.gz
cvid_responses_2020_04_07_recordedby_2020_06_03.csv.gz
cvid_responses_2020_04_11_recordedby_2020_09_03.csv.gz
cvid_responses_20

cvid_responses_2020_07_28_recordedby_2020_07_29.csv.gz
cvid_responses_2020_07_28_recordedby_2020_07_30.csv.gz
cvid_responses_2020_07_28_recordedby_2020_07_31.csv.gz
cvid_responses_2020_07_28_recordedby_2020_08_01.csv.gz
cvid_responses_2020_07_28_recordedby_2020_08_02.csv.gz
cvid_responses_2020_07_28_recordedby_2020_08_03.csv.gz
cvid_responses_2020_07_28_recordedby_2020_09_03.csv.gz
cvid_responses_2020_07_28_recordedby_2020_12_09.csv.gz
cvid_responses_2020_07_29_recordedby_2020_07_30.csv.gz
cvid_responses_2020_07_29_recordedby_2020_07_31.csv.gz
cvid_responses_2020_07_29_recordedby_2020_08_01.csv.gz
cvid_responses_2020_07_29_recordedby_2020_08_02.csv.gz
cvid_responses_2020_07_29_recordedby_2020_08_03.csv.gz
cvid_responses_2020_07_29_recordedby_2020_08_04.csv.gz
cvid_responses_2020_07_29_recordedby_2020_09_03.csv.gz
cvid_responses_2020_07_29_recordedby_2020_12_09.csv.gz
cvid_responses_2020_07_30_recordedby_2020_07_31.csv.gz
cvid_responses_2020_07_30_recordedby_2020_08_01.csv.gz
cvid_respo

## Get all the monthly files ({YYYY}-{MM}.csv.gz) and put them into a .sqlite file

In [3]:
# Check if a file exists, it so, we load it directly
# if not, we will make an engine for querying the data
engine = loadEngine("data/survey.sqlite")

## Total number of samples from all monthly files

In [4]:
query = """select count(*) as num_of_observations from survey"""
num_row = pd.read_sql_query(query, engine)
num_row

,num_of_observations
0,15419328


## Show all column names

In [5]:
# Get a quick view on the data
query = """select * from survey limit 1"""
df = pd.read_sql_query(query, engine)
print(" ".join(df))
df

index StartDatetime EndDatetime A1_1 A1_2 A1_3 A1_4 A1_5 A2 A2b A3 A3b A4 B2 B2_14_TEXT B2b B3 B4 B5 B6 C1 C2 C3 C4 C5 C6 C7 C8_1 C8_2 C9 C10_1_1 C10_2_1 C10_3_1 C10_4_1 C11 C12 D1 D1_4_TEXT D1b D2 D3 D4 D5 Q36 Q40 wave UserLanguage fips weight A5_1 A5_2 A5_3 B2c B2c_14_TEXT B7 B8 B10 B10a B10b B12 B12a B11 C8_3 C13 C13a C14 C15 D8 D9 Q64 Q65 Q66 Q67 Q68 Q69 Q70 Q71 Q72 Q73 Q74 Q75 Q76 Q77 Q78 Q79 Q80 D10 C16 C17 E3 E1_1 E1_2 E1_3 E1_4 E2_1 E2_2 V1 V2 V3 V4_1 V4_2 V4_3 V4_4 V4_5 V9


,index,StartDatetime,EndDatetime,A1_1,A1_2,A1_3,A1_4,A1_5,A2,A2b,...,E2_2,V1,V2,V3,V4_1,V4_2,V4_3,V4_4,V4_5,V9
0,0,2020-04-06,2020-04-06,2.0,2.0,2.0,2.0,2.0,None,6.0,...,None,None,None,None,None,None,None,None,None,None


## Number of reponses by wave

We can see that we have only up to 7 waves and most responses come from wave 3 (May 21 2020- September 7 2020)

In [16]:
# Get a quick view on the data
query = """select count(*) from survey GROUP BY wave"""
view = pd.read_sql_query(query, engine)
view

,count(*)
0,100017
1,1547986
2,7334889
3,2966381
4,1274141
5,1246934
6,948980
